## Analyse de la qualité des données du fichier Marketplace_Import_Mars2025

### 1. Valeurs manquantes (11,25%)
- **45 valeurs manquantes** dans la colonne `statut_retour` (11,25% des données)
- Impact : Ces valeurs doivent être imputées pour permettre une analyse complète du cycle de vie des commandes

### 2. Problème d'uniformisation de la casse
- **Identifiants avec variations de casse** : `mkt-10003` vs `MKT-10003`
- Les identifiants `marketplace_id` présentent des inconsistances de casse qui peuvent causer des problèmes lors des jointures et analyses
- Nombre de valeurs uniques affectées : 10 identifiants différents présentent cette anomalie

### 3. Format de la colonne date
- La colonne `date` est cohérente mais reste de **type object**
- Nécessite une conversion en type `datetime` pour toute analyse temporelle (tendances, saisonnalité, etc.)

### 4. Validation des colonnes catégorielles
- **Colonne `produit`** : 10 produits uniques identifiés
  - Clavier (62 occurrences)
  - Câble USB-C (47 occurrences)
  - Disque SSD (44 occurrences)
  - Écran 24 pouces (42 occurrences)
  - Enceinte Bluetooth (42 occurrences)
  - Etc.

- **Colonne `statut_retour`** : 3 statuts valides (hors valeurs manquantes)
  - Livré (246 occurrences)
  - En attente (70 occurrences)
  - Retourné (39 occurrences)

### Points positifs
- ✅ **Aucun doublon** détecté dans le dataset
- ✅ **Complétude parfaite** pour les colonnes essentielles (`marketplace_id`, `order_id_market`, `date`, `produit`, `quantite`)
- ✅ **Cohérence du format de date** : toutes les dates sont valides et convertibles"""



# Analyse (Laurent)

## 1. Valeurs manquantes dans le champs 'statut_retour'
- la valeur de remplacement est plus explicite en "Inconnu" que "Aucun retour"

## 2. Problème d'uniformisation de la casse
- Il n'y a aucun problème de casse sur les marketplace_id (relire le CSV source)

## 3. Champs date
- les CSV fournissent toujours des types string
- le format des dates est conforme, la conversion en date ne pose pas de problème
- il s'agit probablement de la date de la dernière MàJ du statu
<br> --> il manque les dates de livraison pour tous les produits 

## 4. Le champs produit correspond à sa description
- les doublons sur la description ne sont pas une anomalie
- il faudrait qu'il y ait un champs référence (product_id)
- il faudrait un champs 'price'

## Points oubliés
- ⚠️ il y a des doublons sur les order_id_market
<br/> --> l'ID des commandes est incrémenté par chaque marketplace, des fois pas de bol
- ⚠️ les dates du fichiers ne sont pas cohérentes avec le nom du fichier (uniquement mars 2025)
- ⚠️ Les corrections ne doivent pas être faites sur le fichier source
- L'export a été revu pour ne comporter que les données conformes et cohérentes


## 1 Valeurs manquantes

In [16]:
import pandas as pd
import numpy as np

In [17]:
df_marketplace = pd.read_csv('../data/raw/Marketplace_Imports_Mars2025.csv')

In [18]:
print("AUDIT DATAFRAME MARKETPLACE")

display(df_marketplace.head())

display(df_marketplace.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_marketplace.isnull().sum() / len(df_marketplace) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n---Vérification des doublons---")
print("Nombre total de doublons :", df_marketplace.duplicated().sum())
duplicated_rows = df_marketplace[df_marketplace.duplicated(keep=False)]
print(duplicated_rows.sort_values(by=df_marketplace.columns[0]))


print("\n--- Analyse des Incohérences dans les Colonnes Catégorielles ---")
cols_to_check = ['marketplace_id', 'produit', 'statut_retour']

for col in cols_to_check:
    print(f"\nColonnes : {col}")
    # Compte les valeurs uniques après avoir converti en minuscules (pour identifier les problèmes de casse)
    if df_marketplace[col].dtype == 'object':
        value_counts = df_marketplace[col].str.lower().value_counts(dropna=False)
        print(value_counts.head(10))


# Exemple d'identification des problèmes de casse dans 'marketplace_id'
print("\nExemple de marketplace_id non uniformisé :")
# Si 'mp1' et 'MP1' existent, ils apparaîtront séparément dans df_marketplace['marketplace_id'].value_counts()
print(df_marketplace['marketplace_id'].value_counts(dropna=False).head())
df_marketplace_lowId = df_marketplace['marketplace_id'].value_counts(dropna=False)
print(f"Nombre de marketplace_id non uniformes : {df_marketplace_lowId.count()}")



print("\n--- Analyse de Cohérence de la Colonne Date ---")
try:
    # Tenter la conversion en utilisant 'coerce' pour mettre les dates invalides à NaT (Not a Time)
    df_marketplace['date_temp'] = pd.to_datetime(df_marketplace['date'], errors='coerce')

    # Compter le nombre de dates qui n'ont pas pu être converties (erreurs de format)
    invalid_dates_count = df_marketplace['date_temp'].isna().sum() - df_marketplace['date'].isna().sum()

    if invalid_dates_count > 0:
        print(f"{invalid_dates_count} entrées de la colonne 'date' n'ont pas un format de date valide.")
        # Afficher les valeurs d'origine qui ont échoué la conversion
        invalid_dates = df_marketplace[df_marketplace['date_temp'].isna() & df_marketplace['date'].notna()]['date'].unique()
        print(f"Exemples de formats invalides : {invalid_dates[:5]}")
    else:
        print("La colonne 'date' semble avoir un format cohérent et valide pour la conversion en datetime.")

    df_marketplace.drop(columns=['date_temp'], inplace=True) # Supprimer la colonne temporaire

except Exception as e:
    print(f"Une erreur inattendue est survenue lors de l'analyse de la date : {e}")


AUDIT DATAFRAME MARKETPLACE


,marketplace_id,order_id_market,date,produit,quantite,statut_retour
0,MKT-10001,774,2025-10-20,Écran 24 pouces,5,Livré
1,MKT-10002,1472,2025-05-24,Clavier,1,En attente
2,MKT-10003,1273,2024-11-14,Batterie externe,5,Livré
3,MKT-10004,1539,2024-11-30,Souris,2,Livré
4,MKT-10005,1287,2025-07-21,Clavier,2,Livré


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   marketplace_id   400 non-null    object
 1   order_id_market  400 non-null    int64 
 2   date             400 non-null    object
 3   produit          400 non-null    object
 4   quantite         400 non-null    int64 
 5   statut_retour    355 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


None


---Taux de valeurs manquantes (%)---
statut_retour    11.25
dtype: float64

---Vérification des doublons---
Nombre total de doublons : 0
Empty DataFrame
Columns: [marketplace_id, order_id_market, date, produit, quantite, statut_retour]
Index: []

--- Analyse des Incohérences dans les Colonnes Catégorielles ---

Colonnes : marketplace_id
marketplace_id
mkt-10001    1
mkt-10002    1
mkt-10003    1
mkt-10004    1
mkt-10005    1
mkt-10006    1
mkt-10007    1
mkt-10008    1
mkt-10009    1
mkt-10010    1
Name: count, dtype: int64

Colonnes : produit
produit
clavier               62
câble usb-c           47
disque ssd            44
écran 24 pouces       42
enceinte bluetooth    42
chargeur rapide       41
webcam hd             39
batterie externe      30
casque bluetooth      29
souris                24
Name: count, dtype: int64

Colonnes : statut_retour
statut_retour
livré         246
en attente     70
NaN            45
retourné       39
Name: count, dtype: int64

Exemple de marketplace_id 

## Vérifications contradictoire (Laurent)
L'affichage au-dessus présente la liste après avoir converti des données en minuscule

In [19]:
# Compter les marketplace_id en minuscules
print("\n--- Analyse de la casse des marketplace_id ---")

# Compter combien de valeurs sont déjà entièrement en minuscules
is_lowercase = df_marketplace['marketplace_id'].dropna().str.islower()
nb_lowercase = is_lowercase.sum()
nb_total = len(df_marketplace['marketplace_id'].dropna())

print(f"Nombre de marketplace_id en minuscules : {nb_lowercase} sur {nb_total} ({nb_lowercase/nb_total*100:.2f}%)")
print(f"Nombre de marketplace_id avec majuscules : {nb_total - nb_lowercase} sur {nb_total} ({(nb_total - nb_lowercase)/nb_total*100:.2f}%)")


print("\n--- Vérification des doublons de marketplace_id ---")

# Compter les occurrences de chaque marketplace_id
marketplace_counts = df_marketplace['marketplace_id'].value_counts(dropna=False)

# Identifier ceux qui apparaissent plus d'une fois
duplicated_marketplace_ids = marketplace_counts[marketplace_counts > 1]

if len(duplicated_marketplace_ids) > 0:
    print(f"DOUBLONS DÉTECTÉS : {len(duplicated_marketplace_ids)} marketplace_id apparaissent plusieurs fois")
    print(f"Nombre total d'enregistrements en doublon : {duplicated_marketplace_ids.sum() - len(duplicated_marketplace_ids)}")

    print("\n--- Détail des marketplace_id en doublon ---")
    for marketplace_id, count in duplicated_marketplace_ids.items():
        print(f"  '{marketplace_id}' : {count} occurrences")

        # Afficher quelques exemples de lignes en doublon
        duplicated_rows = df_marketplace[df_marketplace['marketplace_id'] == marketplace_id]
        print(f"    Exemples de lignes concernées (indices) : {duplicated_rows.index.tolist()[:5]}")

    # Afficher un échantillon complet des doublons
    print("\n--- Échantillon des lignes en doublon ---")
    # Prendre le premier marketplace_id en doublon pour exemple
    first_dup_id = duplicated_marketplace_ids.index[0]
    sample_rows = df_marketplace[df_marketplace['marketplace_id'] == first_dup_id]
    display(sample_rows)

else:
    print("✓ Aucun doublon de marketplace_id détecté")
    print(f"Tous les {len(marketplace_counts)} marketplace_id sont uniques")

# Statistiques supplémentaires
print("\n--- Statistiques générales ---")
print(f"Nombre total d'enregistrements : {len(df_marketplace)}")
print(f"Nombre de marketplace_id uniques : {df_marketplace['marketplace_id'].nunique()}")
print(f"Nombre de valeurs manquantes (NaN) : {df_marketplace['marketplace_id'].isna().sum()}")



--- Analyse de la casse des marketplace_id ---
Nombre de marketplace_id en minuscules : 0 sur 400 (0.00%)
Nombre de marketplace_id avec majuscules : 400 sur 400 (100.00%)

--- Vérification des doublons de marketplace_id ---
✓ Aucun doublon de marketplace_id détecté
Tous les 400 marketplace_id sont uniques

--- Statistiques générales ---
Nombre total d'enregistrements : 400
Nombre de marketplace_id uniques : 400
Nombre de valeurs manquantes (NaN) : 0


In [20]:
print("\n--- Vérification des doublons de order_id_market ---")

# Compter les occurrences de chaque order_id_market
order_counts = df_marketplace['order_id_market'].value_counts(dropna=False)

# Identifier ceux qui apparaissent plus d'une fois
duplicated_order_ids = order_counts[order_counts > 1]

if len(duplicated_order_ids) > 0:
    print(f"⚠️ DOUBLONS DÉTECTÉS : {len(duplicated_order_ids)} order_id_market apparaissent plusieurs fois")
    print(f"Nombre total d'enregistrements en doublon : {duplicated_order_ids.sum() - len(duplicated_order_ids)}")

    # CRÉATION D'UN DATAFRAME CONTENANT UNIQUEMENT LES LIGNES EN DOUBLON
    df_marketplace_duplicates = df_marketplace[df_marketplace['order_id_market'].isin(duplicated_order_ids.index)].copy()
    df_marketplace_duplicates = df_marketplace_duplicates.sort_values(by='order_id_market')

    print(f"\nDataFrame créé : df_marketplace_duplicates ({len(df_marketplace_duplicates)} lignes)")

    print("\n--- Détail des order_id_market en doublon ---")
    for order_id, count in duplicated_order_ids.items():
        print(f"  '{order_id}' : {count} occurrences")

        # Afficher quelques exemples de lignes en doublon
        duplicated_rows = df_marketplace_duplicates[df_marketplace_duplicates['order_id_market'] == order_id]
        print(f"    Exemples de lignes concernées (indices) : {duplicated_rows.index.tolist()[:5]}")

    # Afficher un échantillon complet des doublons
    print("\n--- Échantillon des lignes en doublon ---")
    # Prendre le premier order_id_market en doublon pour exemple
    first_dup_id = duplicated_order_ids.index[0]
    sample_rows = df_marketplace_duplicates[df_marketplace_duplicates['order_id_market'] == first_dup_id]
    display(sample_rows)

    # Analyse complémentaire : vérifier si ce sont de vrais doublons ou des lignes différentes
    print("\n--- Analyse de la nature des doublons ---")
    print("Vérification si les lignes en doublon sont identiques ou différentes :")
    for order_id in duplicated_order_ids.index[:3]:  # Analyser les 3 premiers
        dup_rows = df_marketplace_duplicates[df_marketplace_duplicates['order_id_market'] == order_id]
        # Vérifier si toutes les colonnes sont identiques
        if len(dup_rows.drop_duplicates()) == 1:
            print(f"  '{order_id}' : Doublon EXACT (toutes les colonnes identiques)")
        else:
            print(f"  '{order_id}' : Doublon PARTIEL (même order_id mais autres données différentes)")
            # Afficher les différences
            print(f"    Produits concernés : {dup_rows['produit'].unique().tolist()}")
            print(f"    Dates : {dup_rows['date'].unique().tolist()}")

    # ANALYSE : order_id_market avec plusieurs marketplace_id différents
    print("\n--- Analyse des order_id_market avec plusieurs marketplace_id ---")

    # Créer un DataFrame d'analyse groupé par order_id_market
    df_analysis = df_marketplace_duplicates.groupby('order_id_market').agg({
        'marketplace_id': ['nunique', lambda x: list(x.unique())],
        'order_id_market': 'count',
        'produit': lambda x: list(x.unique()),
        'date': lambda x: list(x.unique())
    }).reset_index()

    # Renommer les colonnes pour plus de clarté
    df_analysis.columns = ['order_id_market', 'nb_marketplaces', 'marketplaces_list', 'nb_occurrences', 'produits', 'dates']

    # Filtrer pour ne garder que ceux avec plusieurs marketplace_id
    df_multiple_marketplaces = df_analysis[df_analysis['nb_marketplaces'] > 1].copy()

    if len(df_multiple_marketplaces) > 0:
        print(f"⚠️ ANOMALIE CRITIQUE : {len(df_multiple_marketplaces)} order_id_market ont plusieurs marketplace_id différents !")
        print(f"   (Une commande ne devrait normalement appartenir qu'à UNE seule marketplace)\n")
        print(f"DataFrame créé : df_multiple_marketplaces ({len(df_multiple_marketplaces)} lignes)\n")

        # Afficher les 10 premiers cas
        for idx, row in df_multiple_marketplaces.head(10).iterrows():
            print(f"  Order '{row['order_id_market']}' :")
            print(f"    - Apparaît sur {row['nb_marketplaces']} marketplaces différentes : {row['marketplaces_list']}")
            print(f"    - Nombre total d'occurrences : {row['nb_occurrences']}")

            # Afficher le détail des lignes concernées depuis le df des doublons
            detail_rows = df_marketplace_duplicates[df_marketplace_duplicates['order_id_market'] == row['order_id_market']]
            print(f"    - Détail :")
            for idx_detail, row_detail in detail_rows.iterrows():
                print(f"      {row_detail['marketplace_id']} | {row_detail['produit']} | {row_detail['date']} | Qté: {row_detail['quantite']}")
            print()

        # Statistiques globales sur cette anomalie
        print(f"\n--- Statistiques sur l'anomalie ---")
        print(f"Nombre total d'order_id_market en doublon : {len(duplicated_order_ids)}")
        print(f"Dont avec plusieurs marketplace_id : {len(df_multiple_marketplaces)} ({len(df_multiple_marketplaces)/len(duplicated_order_ids)*100:.1f}%)")
        print(f"Dont sur la même marketplace : {len(duplicated_order_ids) - len(df_multiple_marketplaces)} ({(len(duplicated_order_ids) - len(df_multiple_marketplaces))/len(duplicated_order_ids)*100:.1f}%)")

        # Afficher un aperçu du DataFrame d'analyse
        print("\n--- Aperçu du DataFrame df_multiple_marketplaces ---")
        display(df_multiple_marketplaces.head())

    else:
        print("✓ Tous les order_id_market en doublon appartiennent à la même marketplace")
        print("  (Les doublons sont probablement des commandes multi-produits ou des erreurs d'import)")

else:
    print("✓ Aucun doublon de order_id_market détecté")
    print(f"Tous les {len(order_counts)} order_id_market sont uniques")

# Statistiques supplémentaires
print("\n--- Statistiques générales ---")
print(f"Nombre total d'enregistrements : {len(df_marketplace)}")
print(f"Nombre de order_id_market uniques : {df_marketplace['order_id_market'].nunique()}")
print(f"Nombre de valeurs manquantes (NaN) : {df_marketplace['order_id_market'].isna().sum()}")
print(f"Nombre de marketplace_id uniques : {df_marketplace['marketplace_id'].nunique()}")
print(f"Pourcentage de order_id_market_uniques : {df_marketplace['order_id_market'].nunique()/len(df_marketplace)*100}")


--- Vérification des doublons de order_id_market ---
⚠️ DOUBLONS DÉTECTÉS : 33 order_id_market apparaissent plusieurs fois
Nombre total d'enregistrements en doublon : 34

DataFrame créé : df_marketplace_duplicates (67 lignes)

--- Détail des order_id_market en doublon ---
  '1819' : 3 occurrences
    Exemples de lignes concernées (indices) : [348, 76, 278]
  '38' : 2 occurrences
    Exemples de lignes concernées (indices) : [221, 18]
  '875' : 2 occurrences
    Exemples de lignes concernées (indices) : [22, 217]
  '1194' : 2 occurrences
    Exemples de lignes concernées (indices) : [284, 9]
  '8' : 2 occurrences
    Exemples de lignes concernées (indices) : [8, 296]
  '1042' : 2 occurrences
    Exemples de lignes concernées (indices) : [43, 372]
  '452' : 2 occurrences
    Exemples de lignes concernées (indices) : [362, 382]
  '892' : 2 occurrences
    Exemples de lignes concernées (indices) : [368, 300]
  '140' : 2 occurrences
    Exemples de lignes concernées (indices) : [317, 373]


,marketplace_id,order_id_market,date,produit,quantite,statut_retour
348,MKT-10349,1819,2025-06-18,Souris,2,Retourné
76,MKT-10077,1819,2025-03-02,Clavier,2,Livré
278,MKT-10279,1819,2024-12-24,Clavier,2,Retourné



--- Analyse de la nature des doublons ---
Vérification si les lignes en doublon sont identiques ou différentes :
  '1819' : Doublon PARTIEL (même order_id mais autres données différentes)
    Produits concernés : ['Souris', 'Clavier']
    Dates : ['2025-06-18', '2025-03-02', '2024-12-24']
  '38' : Doublon PARTIEL (même order_id mais autres données différentes)
    Produits concernés : ['Enceinte Bluetooth', 'Écran 24 pouces']
    Dates : ['2025-10-12', '2025-09-05']
  '875' : Doublon PARTIEL (même order_id mais autres données différentes)
    Produits concernés : ['Batterie externe', 'Casque Bluetooth']
    Dates : ['2024-12-07', '2025-05-02']

--- Analyse des order_id_market avec plusieurs marketplace_id ---
⚠️ ANOMALIE CRITIQUE : 33 order_id_market ont plusieurs marketplace_id différents !
   (Une commande ne devrait normalement appartenir qu'à UNE seule marketplace)

DataFrame créé : df_multiple_marketplaces (33 lignes)

  Order '8' :
    - Apparaît sur 2 marketplaces différentes :

,order_id_market,nb_marketplaces,marketplaces_list,nb_occurrences,produits,dates
0,8,2,"[MKT-10009, MKT-10297]",2,"[Câble USB-C, Casque Bluetooth]","[2024-11-16, 2024-12-06]"
1,38,2,"[MKT-10222, MKT-10019]",2,"[Enceinte Bluetooth, Écran 24 pouces]","[2025-10-12, 2025-09-05]"
2,68,2,"[MKT-10308, MKT-10035]",2,"[Clavier, Écran 24 pouces]","[2025-04-01, 2025-01-02]"
3,101,2,"[MKT-10097, MKT-10278]",2,[Webcam HD],"[2025-09-02, 2024-12-09]"
4,111,2,"[MKT-10274, MKT-10220]",2,"[Casque Bluetooth, Câble USB-C]","[2025-08-23, 2025-06-23]"



--- Statistiques générales ---
Nombre total d'enregistrements : 400
Nombre de order_id_market uniques : 366
Nombre de valeurs manquantes (NaN) : 0
Nombre de marketplace_id uniques : 400
Pourcentage de order_id_market_uniques : 91.5


# Vérification de la période de l'export (sensé être mensuel)

In [26]:
print("\n--- Vérification de la cohérence temporelle de l'export ---")

# Convertir les dates si ce n'est pas déjà fait
df_marketplace['date'] = pd.to_datetime(df_marketplace['date'], errors='coerce')

# Identifier la période attendue d'après le nom du fichier
# Marketplace_Imports_Mars2025.csv → Mars 2025
periode_attendue_debut = pd.Timestamp('2025-03-01')
periode_attendue_fin = pd.Timestamp('2025-03-31')

print(f"Période attendue d'après le fichier : Mars 2025")
print(f"  Du {periode_attendue_debut.strftime('%d/%m/%Y')} au {periode_attendue_fin.strftime('%d/%m/%Y')}")

# Analyser les dates réelles présentes dans le fichier
date_min = df_marketplace['date'].min()
date_max = df_marketplace['date'].max()

print(f"\nPériode réelle dans les données :")
print(f"  Date la plus ancienne : {date_min.strftime('%d/%m/%Y') if pd.notna(date_min) else 'NaT'}")
print(f"  Date la plus récente : {date_max.strftime('%d/%m/%Y') if pd.notna(date_max) else 'NaT'}")

# Identifier les lignes hors période
mask_avant = df_marketplace['date'] < periode_attendue_debut
mask_apres = df_marketplace['date'] > periode_attendue_fin
mask_hors_periode = mask_avant | mask_apres

nb_hors_periode = mask_hors_periode.sum()
nb_total = len(df_marketplace)
pourcentage_hors_periode = (nb_hors_periode / nb_total) * 100

if nb_hors_periode > 0:
    print(f"\n⚠️ ANOMALIE DÉTECTÉE : {nb_hors_periode} lignes ({pourcentage_hors_periode:.2f}%) ont des dates hors période !")

    # Détailler les anomalies
    nb_avant = mask_avant.sum()
    nb_apres = mask_apres.sum()

    print(f"  - Dates avant mars 2025 : {nb_avant}")
    print(f"  - Dates après mars 2025 : {nb_apres}")

    # Créer un DataFrame des anomalies
    df_hors_periode = df_marketplace[mask_hors_periode].copy()
    df_hors_periode = df_hors_periode.sort_values('date')

    print("\n--- Exemples de lignes avec dates hors période ---")
    display(df_hors_periode[['marketplace_id', 'order_id_market', 'date', 'produit']].head(10))

    # Analyse par mois
    print("\n--- Répartition par mois/année ---")
    df_marketplace['annee_mois'] = df_marketplace['date'].dt.to_period('M')
    repartition = df_marketplace['annee_mois'].value_counts().sort_index()
    print(repartition)

    # Nettoyer la colonne temporaire
    df_marketplace.drop(columns=['annee_mois'], inplace=True)

else:
    print(f"\n✓ Toutes les dates correspondent à la période attendue (Mars 2025)")

# Vérifier les dates NaT (non converties)
nb_nat = df_marketplace['date'].isna().sum()
if nb_nat > 0:
    print(f"\n⚠️ {nb_nat} dates invalides (NaT) détectées")

# EXTRACTION DU Q1 2025
print("\n--- Extraction des données du 1er trimestre 2025 ---")

# Définir la période Q1 2025
q1_debut = pd.Timestamp('2025-01-01')
q1_fin = pd.Timestamp('2025-03-31')

print(f"Période Q1 2025 : Du {q1_debut.strftime('%d/%m/%Y')} au {q1_fin.strftime('%d/%m/%Y')}")

# Filtrer les données du Q1 2025
mask_q1 = (df_marketplace['date'] >= q1_debut) & (df_marketplace['date'] <= q1_fin)
df_marketplace_q1 = df_marketplace[mask_q1].copy()

nb_lignes_q1 = len(df_marketplace_q1)
pourcentage_q1 = (nb_lignes_q1 / nb_total) * 100

print(f"\nNombre de lignes correspondant au Q1 2025 : {nb_lignes_q1} ({pourcentage_q1:.2f}%)")
print(f"Nombre de lignes hors Q1 2025 : {nb_total - nb_lignes_q1} ({100 - pourcentage_q1:.2f}%)")

if nb_lignes_q1 > 0:
    # Statistiques sur le Q1
    print("\n--- Statistiques sur les données Q1 2025 ---")
    print(f"  Date minimale : {df_marketplace_q1['date'].min().strftime('%d/%m/%Y')}")
    print(f"  Date maximale : {df_marketplace_q1['date'].max().strftime('%d/%m/%Y')}")
    print(f"  Nombre de marketplaces : {df_marketplace_q1['marketplace_id'].nunique()}")
    print(f"  Nombre de commandes uniques : {df_marketplace_q1['order_id_market'].nunique()}")

    # CORRECTION DES STATUTS RETOUR MANQUANTS
    print("\n--- Correction des statuts_retour manquants ---")
    nb_statuts_manquants = df_marketplace_q1['statut_retour'].isnull().sum()

    if nb_statuts_manquants > 0:
        print(f"Nombre de statuts_retour manquants : {nb_statuts_manquants} ({nb_statuts_manquants/nb_lignes_q1*100:.2f}%)")
        df_marketplace_q1['statut_retour'] = df_marketplace_q1['statut_retour'].fillna('Inconnu')
        print(f"✓ {nb_statuts_manquants} valeurs remplacées par 'Inconnu'")
    else:
        print("✓ Aucun statut_retour manquant")

    # Vérification après correction
    print(f"Valeurs manquantes restantes : {df_marketplace_q1['statut_retour'].isnull().sum()}")

    # Afficher la répartition des statuts après correction
    print("\n--- Répartition des statuts_retour après correction ---")
    print(df_marketplace_q1['statut_retour'].value_counts())

    # Aperçu du DataFrame exporté
    print("\n--- Aperçu des données Q1 2025 après correction ---")
    display(df_marketplace_q1.head(10))

    # SAUVEGARDE
    print("\n--- SAUVEGARDE DU FICHIER Q1 2025 ---")
    df_marketplace_q1.to_csv('../data/processed/Marketplace_Export_Q1_2025.csv', index=False)
    print(f"✓ Fichier sauvegardé : ../data/processed/Marketplace_Export_Q1_2025.csv ({nb_lignes_q1} lignes)")

else:
    print("\n⚠️ Aucune donnée trouvée pour le Q1 2025 !")


--- Vérification de la cohérence temporelle de l'export ---
Période attendue d'après le fichier : Mars 2025
  Du 01/03/2025 au 31/03/2025

Période réelle dans les données :
  Date la plus ancienne : 29/10/2024
  Date la plus récente : 28/10/2025

⚠️ ANOMALIE DÉTECTÉE : 368 lignes (92.00%) ont des dates hors période !
  - Dates avant mars 2025 : 131
  - Dates après mars 2025 : 237

--- Exemples de lignes avec dates hors période ---


,marketplace_id,order_id_market,date,produit
144,MKT-10145,1489,2024-10-29,Casque Bluetooth
356,MKT-10357,1487,2024-10-29,Disque SSD
121,MKT-10122,1279,2024-10-31,Casque Bluetooth
284,MKT-10285,1194,2024-10-31,Batterie externe
223,MKT-10224,1437,2024-11-02,Écran 24 pouces
108,MKT-10109,483,2024-11-04,Clavier
275,MKT-10276,1990,2024-11-04,Batterie externe
80,MKT-10081,702,2024-11-05,Webcam HD
395,MKT-10396,1500,2024-11-07,Disque SSD
71,MKT-10072,515,2024-11-07,Webcam HD



--- Répartition par mois/année ---
annee_mois
2024-10     4
2024-11    38
2024-12    30
2025-01    35
2025-02    24
2025-03    32
2025-04    27
2025-05    39
2025-06    31
2025-07    31
2025-08    25
2025-09    45
2025-10    39
Freq: M, Name: count, dtype: int64

--- Extraction des données du 1er trimestre 2025 ---
Période Q1 2025 : Du 01/01/2025 au 31/03/2025

Nombre de lignes correspondant au Q1 2025 : 91 (22.75%)
Nombre de lignes hors Q1 2025 : 309 (77.25%)

--- Statistiques sur les données Q1 2025 ---
  Date minimale : 01/01/2025
  Date maximale : 30/03/2025
  Nombre de marketplaces : 91
  Nombre de commandes uniques : 91

--- Correction des statuts_retour manquants ---
Nombre de statuts_retour manquants : 11 (12.09%)
✓ 11 valeurs remplacées par 'Inconnu'
Valeurs manquantes restantes : 0

--- Répartition des statuts_retour après correction ---
statut_retour
Livré         58
En attente    16
Inconnu       11
Retourné       6
Name: count, dtype: int64

--- Aperçu des données Q1 2025

,marketplace_id,order_id_market,date,produit,quantite,statut_retour
6,MKT-10007,1851,2025-02-03,Écran 24 pouces,1,Livré
7,MKT-10008,696,2025-02-14,Disque SSD,2,Livré
9,MKT-10010,1194,2025-01-06,Enceinte Bluetooth,1,Livré
12,MKT-10013,1532,2025-03-13,Batterie externe,3,Livré
16,MKT-10017,407,2025-03-11,Enceinte Bluetooth,5,Livré
17,MKT-10018,1905,2025-01-30,Chargeur rapide,1,Livré
19,MKT-10020,474,2025-03-26,Disque SSD,2,Inconnu
23,MKT-10024,1277,2025-01-14,Souris,2,En attente
30,MKT-10031,842,2025-01-20,Batterie externe,3,Livré
31,MKT-10032,484,2025-03-25,Enceinte Bluetooth,2,Livré



--- SAUVEGARDE DU FICHIER Q1 2025 ---
✓ Fichier sauvegardé : ../data/processed/Marketplace_Export_Q1_2025.csv (91 lignes)


In [22]:
print("\n--- Analyse Approfondie des Colonnes 'produit' et 'statut_retour' ---")
# 1. Analyse de la colonne 'produit'
print("\n Colonne : produit (TOP 12 Libellés après uniformisation de la casse)")
# Utilisation de .str.lower() pour regrouper les variations de casse (ex: 'A' vs 'a')
valeurs_produit = df_marketplace['produit'].str.lower().value_counts()
print(valeurs_produit.head(12))
print(f"Nombre total de produits uniques : {len(valeurs_produit)}")

# 2. Analyse de la colonne 'statut_retour'
print("\n Colonne : statut_retour (Tous les Libellés après uniformisation de la casse)")
# Utilisation de .str.lower() pour regrouper les variations de casse (ex: 'Ok' vs 'OK')
valeurs_statut = df_marketplace['statut_retour'].str.lower().value_counts()
print(valeurs_statut)
print(f"Nombre total de statuts uniques (N'incluant les valeurs manquantes) : {len(valeurs_statut)}")



--- Analyse Approfondie des Colonnes 'produit' et 'statut_retour' ---

 Colonne : produit (TOP 12 Libellés après uniformisation de la casse)
produit
clavier               62
câble usb-c           47
disque ssd            44
écran 24 pouces       42
enceinte bluetooth    42
chargeur rapide       41
webcam hd             39
batterie externe      30
casque bluetooth      29
souris                24
Name: count, dtype: int64
Nombre total de produits uniques : 10

 Colonne : statut_retour (Tous les Libellés après uniformisation de la casse)
statut_retour
livré         246
en attente     70
retourné       39
Name: count, dtype: int64
Nombre total de statuts uniques (N'incluant les valeurs manquantes) : 3


## Problèmes et solutions proposées

### 45 Valeurs manquantes dans la colonne statut_retour
Les 45 valeurs manquantes doivent être imputées pour être utilisables en analyse. Nous utilisons la catégorie "Aucun retour".

### Identifiants avec des variations de casse (ex: mkt-10003 vs. MKT-10003)
Uniformisation de la casse

### La colonne date est cohérente mais reste de type object
La conversion est nécessaire pour toute analyse temporelle.

In [23]:
# print("--- 1. Correction de la Casse pour 'marketplace_id' ---")
# # Conversion de tous les identifiants en MAJUSCULES (ou minuscules, mais l'uniformisation est clé)
# df_marketplace['marketplace_id'] = df_marketplace['marketplace_id'].str.upper()
# print("La colonne 'marketplace_id' est uniformisée en majuscules.")

# print("\n--- 2. Imputation des Valeurs Manquantes dans 'statut_retour' ---")
# # Remplacement des NaN par 'Aucun retour'
# df_marketplace['statut_retour'] = df_marketplace['statut_retour'].fillna('Aucun retour')
# print(f"La colonne 'statut_retour' est imputée. NaNs restants : {df_marketplace['statut_retour'].isnull().sum()}")

# print("\n--- 3. Conversion de 'date' en 'datetime' ---")
# # Conversion du type object en datetime64
# df_marketplace['date'] = pd.to_datetime(df_marketplace['date'])
# print("La colonne 'date' est convertie au format datetime.")

# # Vérification finale des types de données
# print("\n--- Vérification finale des types ---")
# print(df_marketplace.info())

# print("\n--- Vérification des doublons après nettoyage ---")
# display(df_marketplace.head(12))